In [29]:
import re
import os

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime as dt

import warnings
warnings.filterwarnings("ignore")

In [4]:
# File Paths
data_path = "../Accepted_Data/2018_Accepted_data/2018_Accepted_data.csv"

In [5]:
# reading in csv and converting to dataframes
data = pd.read_csv(data_path, error_bad_lines=False,low_memory=False)

In [6]:
data.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,hardship_payoff_balance_amount,hardship_last_payment_amount,disbursement_method,debt_settlement_flag,debt_settlement_flag_date,settlement_status,settlement_date,settlement_amount,settlement_percentage,settlement_term
0,130954621,NaN,5000.0,5000.0,5000.0,36 months,20.39,186.82,D,D4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
1,130964697,NaN,15000.0,15000.0,15000.0,36 months,9.92,483.45,B,B2,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
2,130955326,NaN,11200.0,11200.0,11200.0,60 months,30.79,367.82,G,G1,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
3,130504052,NaN,25000.0,25000.0,25000.0,60 months,21.85,688.35,D,D5,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN
4,130956066,NaN,3000.0,3000.0,3000.0,36 months,7.34,93.10,A,A4,...,NaN,NaN,Cash,N,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
data.shape

(495242, 151)

#### the dataset I used is derived from the Lending Club for all of 2018. It contains more than 495,000 original loan data of users with 151 attributes.

In [39]:
# selecting only individual applicants - We'll focus only on individual loans
ind_data = data[data['application_type']=='Individual']

In [40]:
ind_data.shape

(426257, 151)

In [41]:
# variables to use
useful_data = ind_data[[
    'loan_amnt',
    'installment',
    'annual_inc',
    'grade',
    'open_acc',
    'total_pymnt',
    'total_rec_int',
    'home_ownership',
    'verification_status',
    'application_type',
    'purpose',
    'term',
    'loan_status'
]]

In [42]:
useful_data.head()

,loan_amnt,installment,annual_inc,grade,open_acc,total_pymnt,total_rec_int,home_ownership,verification_status,application_type,purpose,term,loan_status
0,5000.0,186.82,50000.0,D,5.0,2043.690000,824.00,RENT,Verified,Individual,other,36 months,Current
1,15000.0,483.45,196000.0,B,19.0,5301.420000,1179.92,OWN,Source Verified,Individual,debt_consolidation,36 months,Current
3,25000.0,688.35,65000.0,D,7.0,7511.160000,4699.89,MORTGAGE,Source Verified,Individual,debt_consolidation,60 months,Current
4,3000.0,93.10,52000.0,A,7.0,3011.577285,11.58,RENT,Source Verified,Individual,major_purchase,36 months,Fully Paid
5,17000.0,454.10,52000.0,D,9.0,4956.590000,2976.36,RENT,Source Verified,Individual,debt_consolidation,60 months,Current


In [43]:
# checking data for null values
useful_data.isna().any()

loan_amnt              False
installment            False
annual_inc             False
grade                  False
open_acc               False
total_pymnt            False
total_rec_int          False
home_ownership         False
verification_status    False
application_type       False
purpose                False
term                   False
loan_status            False
dtype: bool

#### Create new variables/features to consider

In [44]:
# monthly income
useful_data['monthly_inc'] = useful_data['annual_inc']/12

In [45]:
useful_data.head()

,loan_amnt,installment,annual_inc,grade,open_acc,total_pymnt,total_rec_int,home_ownership,verification_status,application_type,purpose,term,loan_status,monthly_inc
0,5000.0,186.82,50000.0,D,5.0,2043.690000,824.00,RENT,Verified,Individual,other,36 months,Current,4166.666667
1,15000.0,483.45,196000.0,B,19.0,5301.420000,1179.92,OWN,Source Verified,Individual,debt_consolidation,36 months,Current,16333.333333
3,25000.0,688.35,65000.0,D,7.0,7511.160000,4699.89,MORTGAGE,Source Verified,Individual,debt_consolidation,60 months,Current,5416.666667
4,3000.0,93.10,52000.0,A,7.0,3011.577285,11.58,RENT,Source Verified,Individual,major_purchase,36 months,Fully Paid,4333.333333
5,17000.0,454.10,52000.0,D,9.0,4956.590000,2976.36,RENT,Source Verified,Individual,debt_consolidation,60 months,Current,4333.333333


#### Using Monthly income, create 'Installment_risk' - which is a rate calculated as such: installment_risk = installment/monthly_inc. The higher the number, the higher the risk of an applicant defaulting

In [46]:
useful_data['installment_risk'] = useful_data['installment']/useful_data['monthly_inc']

In [47]:
useful_data.head()

,loan_amnt,installment,annual_inc,grade,open_acc,total_pymnt,total_rec_int,home_ownership,verification_status,application_type,purpose,term,loan_status,monthly_inc,installment_risk
0,5000.0,186.82,50000.0,D,5.0,2043.690000,824.00,RENT,Verified,Individual,other,36 months,Current,4166.666667,0.044837
1,15000.0,483.45,196000.0,B,19.0,5301.420000,1179.92,OWN,Source Verified,Individual,debt_consolidation,36 months,Current,16333.333333,0.029599
3,25000.0,688.35,65000.0,D,7.0,7511.160000,4699.89,MORTGAGE,Source Verified,Individual,debt_consolidation,60 months,Current,5416.666667,0.127080
4,3000.0,93.10,52000.0,A,7.0,3011.577285,11.58,RENT,Source Verified,Individual,major_purchase,36 months,Fully Paid,4333.333333,0.021485
5,17000.0,454.10,52000.0,D,9.0,4956.590000,2976.36,RENT,Source Verified,Individual,debt_consolidation,60 months,Current,4333.333333,0.104792


#### creating final working dataframe

In [48]:
final_df = useful_data

In [51]:
# values in loan status
final_df['loan_status'].unique()

array(['Current', 'Fully Paid', 'Late (31-120 days)', 'Late (16-30 days)',
       'Charged Off', 'Default', 'In Grace Period'], dtype=object)

In [ ]:
# bucket them into 2 categories, Normal and Default
# 'Current', 'Fully Paid' = Normal
# 'Late (31-120 days)', 'Late (16-30 days)', 'Charged Off', 'Default', 'In Grace Period' = Default

In [52]:
# Create conditions for new column
conditions = [
    (final_df['loan_status']=='Current') | (final_df['loan_status']=='Fully Paid'),
    (final_df['loan_status']=='Late (31-120 days)') | (final_df['loan_status']=='Late (16-30 days)') | (final_df['loan_status']=='Charged Off') | (final_df['loan_status']=='Default') | (final_df['loan_status']=='In Grace Period'),
]

In [53]:
# Create values for the conditions
values = ['Normal','Default']

In [54]:
final_df['loan_category'] = np.select(conditions, values)

In [56]:
final_df

,loan_amnt,installment,annual_inc,grade,open_acc,total_pymnt,total_rec_int,home_ownership,verification_status,application_type,purpose,term,loan_status,monthly_inc,installment_risk,loan_category
0,5000.0,186.82,50000.0,D,5.0,2043.690000,824.00,RENT,Verified,Individual,other,36 months,Current,4166.666667,0.044837,Normal
1,15000.0,483.45,196000.0,B,19.0,5301.420000,1179.92,OWN,Source Verified,Individual,debt_consolidation,36 months,Current,16333.333333,0.029599,Normal
3,25000.0,688.35,65000.0,D,7.0,7511.160000,4699.89,MORTGAGE,Source Verified,Individual,debt_consolidation,60 months,Current,5416.666667,0.127080,Normal
4,3000.0,93.10,52000.0,A,7.0,3011.577285,11.58,RENT,Source Verified,Individual,major_purchase,36 months,Fully Paid,4333.333333,0.021485,Normal
5,17000.0,454.10,52000.0,D,9.0,4956.590000,2976.36,RENT,Source Verified,Individual,debt_consolidation,60 months,Current,4333.333333,0.104792,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495237,12000.0,398.46,60000.0,B,16.0,3570.170000,959.71,MORTGAGE,Not Verified,Individual,major_purchase,36 months,Charged Off,5000.000000,0.079692,Default
495238,3000.0,112.09,72000.0,D,11.0,1117.500000,457.68,RENT,Not Verified,Individual,other,36 months,Late (16-30 days),6000.000000,0.018682,Default
495239,14000.0,492.27,52874.0,C,9.0,5402.520000,1808.79,MORTGAGE,Not Verified,Individual,debt_consolidation,36 months,Current,4406.166667,0.111723,Normal
495240,7500.0,245.19,126000.0,B,16.0,2688.010000,649.38,RENT,Not Verified,Individual,other,36 months,Current,10500.000000,0.023351,Normal
